In [ ]:
using Pkg; Pkg.activate(".")

In [ ]:
using LinearAlgebra
using Random
using Distributions
using ForwardDiff

using Plots

Random.seed!(12345)

In [ ]:
include("nbstyle.jl")
include("util.jl")

$$
\newcommand{\N}{\mathcal{N}}
\renewcommand{\vec}[1]{\boldsymbol{#1}}          % vector
\newcommand{\mat}[1]{\boldsymbol{#1}}  
\newcommand{\g}{\mid}
$$

## Build the state-space model
for states and measurements
$$
\vec{x}_k = \begin{pmatrix}\theta_k \\ \dot{\theta}_k\end{pmatrix}\in \mathbb{R}^D \qquad \vec{y}_k = \hat{\theta}_k\in \mathbb{R}^d \qquad k = 1, \dots, T
$$

In [ ]:
d, D = 1, 2;

### Initial distribution

$$\boldsymbol{x}_0 \sim \N(\vec{\mu}_0, \mat{\Sigma}_0)$$
with
$$
\begin{align}
\vec\mu_0 &= \left[0 ~~ -3\right]^\top\\
\mat\Sigma_0 &= \begin{pmatrix}0.1 & 0.0 \\ 0.0 & 1.0\end{pmatrix}
\end{align}
$$

In [ ]:
μ₀ = [0.0, -3.0]
Σ₀ = [0.1 0.0
      0.0 1.0];

### Dynamics
$$\boldsymbol{x}_k \mid \boldsymbol{x}_{k-1} \sim \N(f(\vec{x}_{k-1}), \mat{Q}_{k-1})$$
with
$$
\begin{align}
f(\vec{x}_k) = f\left(\begin{pmatrix}\theta_k \\ \dot\theta_k\end{pmatrix}\right) &= 
\begin{pmatrix}
\theta_k + \dot\theta_k \cdot dt \\
\dot\theta_k + \left(-\alpha \cdot \dot\theta_k - \frac{g}{L} \cdot \sin(\theta_k)\right) \cdot dt
\end{pmatrix}\\[2mm]
    \mat{Q}_k = \mat{Q} &=
    \begin{pmatrix}
        σ_q \cdot dt^3/3 &  σ_q \cdot dt^2/2\\
         σ_q \cdot dt^2/2 &  σ_q \cdot dt
    \end{pmatrix}
\end{align}
$$
and
$$
\begin{align}
    dt &= \lvert t_{k+1} - t_k \rvert\\
    σ_q &= 0.4\\
    g &\approx 9.81 \qquad \tag{gravitational constant} \\
    \alpha &= 0.3 \tag{air resistance}\\
    L &= 3.0 \tag{rod length}
\end{align}
$$

In [ ]:
dt = 0.05
g = 9.81
σ_q = 0.4
air_resistance = 0.3
rod_length = 3.0

function f(x)
    angle, angular_velocity = x
    next_angle = angle + angular_velocity * dt
    next_angular_velocity = angular_velocity + (-air_resistance * angular_velocity - (g / rod_length) * sin(angle)) * dt
    return [next_angle, next_angular_velocity]
end

Q = [σ_q * dt^3/3   σ_q * dt^2/2
     σ_q * dt^2/2   σ_q * dt];

### Measurement model

$$\boldsymbol{y}_k \mid \boldsymbol{x}_k \sim \N(h(\vec{x}_k), \mat{R}_k)$$
with
$$
\begin{align}
h(\vec{x}_k) = h\left(\begin{pmatrix}\theta_k\\\dot\theta_k\end{pmatrix}\right) &=
\theta_k\\[2mm]
    \mat{R}_k = \sigma_r &= 0.08
\end{align}
$$

In [ ]:
function h(x)
    angle, angular_velocity = x
    return angle
end

σ_r = 0.08;

## Simulate a trajectory and draw noisy observations

In [ ]:
ground_truth, observations = simulate_nonlinear(f, Q, h, σ_r, μ₀, Σ₀, 300, rng=MersenneTwister(12));
state_idcs = 1:length(ground_truth)
data_idcs = unique(rand(state_idcs[1:end-1], 15)); 

In [ ]:
T = 0.0:dt:300*dt
plot(
    T, 
    [y[1] for y in ground_truth];
    label="True Location",
    xlabel="time",
    ylabel="angle",
    legend=:bottomright,
    gt_args...
)
scatter!(
    T[data_idcs], 
    [y[1] for y in observations[data_idcs]];
    label="Measurements",
    data_args...
)

## Filtering — The extended Kalman Filter

### Prediction
$$
\begin{align}
      \vec{\mu}^- & = f(\vec{\mu}_{k-1}) \\
      \mat{\Sigma}^- &= \mat{J}_f(\vec{\mu}_{k-1})\mat{\Sigma}_{k-1}\mat{J}_f(\vec{\mu}_{k-1})^\top + \mat{Q}_{k-1}
\end{align}
$$

In [ ]:
function ekf_predict(μ, Σ, f, Q)
    μ⁻ = f(μ)
    Jf = ForwardDiff.jacobian(f, μ)
    Σ⁻ = Jf * Σ * Jf' + Q
    return μ⁻, Σ⁻
end

### Correction
$$
\begin{align}
  \hat{\vec{y}}_k &= h(\vec{\mu}^-_k) \\
  \mat{S}_k &= {\mat{J}_h(\vec{\mu}_k^-)}\mat{\Sigma}^-_k{\mat{J}_h(\vec{\mu}_k^-)^\top} + \mat{R}_k \\
  \mat{K}_k &= \mat{\Sigma}^-_k{\mat{J}_h(\vec{\mu}_k^-)^\top}\mat{S}^{-1}_k \\
  \vec{\mu}_k &= \vec{\mu}^-_k + \mat{K}_k (\vec{y}_k - \hat{\vec{y}}_k)\\
  \mat{\Sigma}_k &= \mat{\Sigma}^-_k - \mat{K}_k\mat{S}_k\mat{K}_k^\top
\end{align}
$$

In [ ]:
function ekf_correct(μ⁻, Σ⁻, h, R, y)
    Jh = ForwardDiff.gradient(h, μ⁻)'
    y_hat = h(μ⁻)
    S = Jh * Σ⁻ * Jh' + R
    K = Σ⁻ * Jh' / S
    μ = μ⁻ + K * (y - y_hat)
    Σ = Σ⁻ - K * S * K'
    return μ, Σ
end

In [ ]:
filter_estimate = [(μ₀, Σ₀)]
predicted_moments = []
for k in state_idcs[2:end]
    # start from last point
    μ, Σ = filter_estimate[end]
    # predict
    μ⁻, Σ⁻ = ekf_predict(μ, Σ, f, Q)
    push!(predicted_moments, (μ⁻, Σ⁻))
    # if there's data: correct
    if k ∈ data_idcs
        push!(filter_estimate, ekf_correct(μ⁻, Σ⁻, h, σ_r, observations[k]))
    else
        push!(filter_estimate, (μ⁻, Σ⁻))
    end
end

In [ ]:
filt_anim_parts = []
filt_anim = @animate for i in 1:length(filter_estimate)
    frame = plot(
        T, 
        [y[1] for y in ground_truth];
        label="True Location",
        legend=:bottomright,
        gt_args...
    )
    scatter!(
        frame, 
        T[data_idcs], 
        [y[1] for y in observations[data_idcs]];
        label="Measurements",
        data_args...
    )
    plot!(
        frame,
        T[1:i], 
        [y[1] for (y, s) in filter_estimate[1:i]];
        ribbon=[2sqrt.(s[1,1]) for (y, s) in filter_estimate[1:i]],
        label="Filter Estimate",
        filter_estimate_args...
        
    )
    
    if i < length(filter_estimate)
        predicted_future = [filter_estimate[i]]
        for (j, _t) in enumerate(T[i+1:end])
            m, S = predicted_future[end]
            push!(
                predicted_future,
                ekf_predict(m, S, f, Q)
            )
        end
        plot!(
            frame,
            T[i:end],
            [y[1] for (y, s) in predicted_future]; 
            ribbon=[2sqrt.(s[1,1]) for (y, s) in predicted_future], 
            label="Predicted estimate",
            prediction_estimate_args...
        )

    end
    
    push!(filt_anim_parts, frame)
end
gif(filt_anim, "out/filt_pendulum.gif", fps=20)

In [ ]:
plot(
    T, 
    [y[1] for (y, s) in filter_estimate];
    ribbon=[2sqrt.(s[1, 1]) for (y, s) in filter_estimate],
    xlabel="time",
    ylabel="angle",
    label="Filter Estimate",
    legend=:bottomright,
    filter_estimate_args...
)
plot!(
    T, 
    [y[1] + 2sqrt(s[1,1]) for (y, s) in filter_estimate];
    label=nothing,
    filter_cred_interval_args...
)
plot!(
    T, 
    [y[1] - 2sqrt(s[1,1]) for (y, s) in filter_estimate];
    label=nothing,
    filter_cred_interval_args...
)
plot!(T, 
    [y[1] for y in ground_truth]; 
    label="True Location",
    gt_args...
)
scatter!(
    T[data_idcs], 
    [y[1] for y in observations[data_idcs]]; 
    label="Measurements", 
    data_args...
)


## Smoothing — The Rauch-Tung-Striebel Smoother

$$
\begin{align}
\mat{G}_{k} &= \mat{\Sigma}_{k}\mat{J}_f(\vec{\mu}_k)^\top \left[\mat{\Sigma}^-_{k+1}\right]^{-1} \\
\vec{\xi}_k &= \vec{\mu}_k + \mat{G}_{k} \left(\vec{\xi}_{k+1} - \vec{\mu}^-_{k+1}\right)\\
\mat{\Lambda}_{k} &= \mat{\Sigma}_{k} + \mat{G}_{k}\left(\mat{\Lambda}_{k+1} - \mat{\Sigma}^-_{k+1}\right)\mat{G}_k^\top
\end{align}
$$

In [ ]:
function ekf_smooth(μ, Σ, ξ_next, Λ_next, μ⁻, Σ⁻, f, Q)
    Jf = ForwardDiff.jacobian(f, μ)    
    G = Σ * Jf' / Symmetric(Σ⁻)
    ξ = μ + G * (ξ_next - μ⁻)
    Λ = Σ + G * (Λ_next - Σ⁻) * G'
    return ξ, Λ
end

In [ ]:
smoother_estimate = [filter_estimate[end]]
for k in reverse(state_idcs[1:end-1])
    ξ_next, Λ_next = smoother_estimate[1]
    μ⁻, Σ⁻ = predicted_moments[k]
    μ, Σ = filter_estimate[k]
    pushfirst!(smoother_estimate, ekf_smooth(μ, Σ, ξ_next, Λ_next, μ⁻, Σ⁻, f, Q))
end

In [ ]:
filt2smooth_anim_parts = []
filt2smooth_anim = @animate for i in reverse(2:length(smoother_estimate)-1)
    frame = plot(
        T, 
        [y[1] for y in ground_truth];
        xlabel="time",
        ylabel="angle",
        label="True Location",
        legend=:bottomright,
        gt_args...
    )
    
    plot!(
        frame,
        T[1:i], 
        [y[1] for (y, s) in filter_estimate[1:i]];
        ribbon=[2*sqrt.(s[1, 1]) for (y, s) in filter_estimate[1:i]],
        label="Filter estimate",
        filter_estimate_args...
    )
    plot!(
        frame,
        T[i-1:end], 
        [y[1] for (y, s) in pushfirst!(deepcopy(smoother_estimate[i:end]), filter_estimate[i-1])];
        ribbon=[2sqrt.(s[1, 1]) for (y, s) in pushfirst!(deepcopy(smoother_estimate[i:end]), filter_estimate[i-1])],
        label="Smoother estimate",
        smoother_estimate_args...
    )
    scatter!(
        frame, 
        T[data_idcs],
        [y[1] for y in observations[data_idcs]];
        label="Measurements",
        data_args...
    )
    push!(filt2smooth_anim_parts, frame)
end
gif(filt2smooth_anim, "out/filt2smooth_anim_pendulum.gif", fps=20)

In [ ]:
plot(
    T, 
    [y[1] for (y, s) in smoother_estimate];
    ribbon=[2sqrt.(s[1, 1]) for (y, s) in smoother_estimate],
    label="Smoother Estimate",
    xlabel="time",
    ylabel="angle",
    legend=:bottomright,
    smoother_estimate_args...
)
plot!(
    T, 
    [y[1] for y in ground_truth]; 
    label="True Location",
    gt_args...
)
scatter!(
    T[data_idcs], 
    [y[1] for y in observations[data_idcs]]; 
    label="Measurements",
    data_args...
)